In [2]:
pip install numpy tensorflow keras opencv-python pillow

Note: you may need to restart the kernel to use updated packages.


In [13]:
from PIL import Image, ImageFile
import os

# Ajustar los límites de tamaño para evitar el error DecompressionBombError
Image.MAX_IMAGE_PIXELS = None
ImageFile.LOAD_TRUNCATED_IMAGES = True

def convert_tiff_to_png(input_path, output_path):
    with Image.open(input_path) as img:
        img.save(output_path, 'PNG')

# Convertir imagen RGB
convert_tiff_to_png('spatial_data/merged_raster_preprocessed.tiff', 'spatial_data/merged_raster_preprocessed.png')

# Convertir imagen de clases
convert_tiff_to_png('spatial_data/badajoz_multiclass_mask.tif', 'spatial_data/badajoz_multiclass_mask.png')


In [14]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate

def crop_to_quarter(image):
    # Obtener las dimensiones de la imagen
    height, width = image.shape[:2]
    # Cortar la imagen al primer cuarto (superior izquierda)
    return image[:height // 2, :width // 2]

# Cargar imagen RGB (3 canales)
img_rgb = cv2.imread('spatial_data/merged_raster_preprocessed.png')
img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
img_rgb = crop_to_quarter(img_rgb)

# Cargar imagen de clases (1 canal)
img_classes = cv2.imread('spatial_data/badajoz_multiclass_mask.png', cv2.IMREAD_GRAYSCALE)
img_classes = crop_to_quarter(img_classes)


In [15]:
# Normalizar imagen RGB
img_rgb = img_rgb / 255.0

# Convertir las clases a formato one-hot
num_classes = 6
img_classes_one_hot = tf.keras.utils.to_categorical(img_classes, num_classes)


In [16]:
print(f"img_rgb shape: {img_rgb.shape}")
print(f"img_classes_one_hot shape: {img_classes_one_hot.shape}")


img_rgb shape: (8640, 11542, 3)
img_classes_one_hot shape: (8640, 11542, 6)


In [19]:
# Aplanar las imágenes para facilitar la división
img_rgb_flat = img_rgb.reshape((img_rgb.shape[0], -1, 3))
img_classes_flat = img_classes_one_hot.reshape((img_classes_one_hot.shape[0], -1, num_classes))

print(f"img_rgb_flat shape: {img_rgb_flat.shape}")
print(f"img_classes_flat shape: {img_classes_flat.shape}")




img_rgb_flat shape: (8640, 11542, 3)
img_classes_flat shape: (8640, 11542, 6)


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(img_rgb_flat, img_classes_flat, test_size=0.2, random_state=42)


In [27]:
def unet_model(input_size=(img_rgb.shape[1], img_rgb.shape[2], 3), num_classes=6):
    inputs = Input(input_size)
    
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    
    # Decoder
    up6 = UpSampling2D(size=(2, 2))(conv5)
    up6 = concatenate([up6, conv4])
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    up7 = concatenate([up7, conv3])
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = UpSampling2D(size=(2, 2))(conv7)
    up8 = concatenate([up8, conv2])
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = UpSampling2D(size=(2, 2))(conv8)
    up9 = concatenate([up9, conv1])
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
    
    conv10 = Conv2D(num_classes, 1, activation='softmax')(conv9)
    
    model = Model(inputs=[inputs], outputs=[conv10])
    
    return model


In [28]:
model = unet_model(input_size=(img_rgb.shape[1], img_rgb.shape[2], 3), num_classes=num_classes)
model.compile(optimizerS='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=16)


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 2884, 0, 512), (None, 2885, 0, 256)]

In [ ]:
scores = model.evaluate(X_val, y_val, verbose=1)
print(f"Accuracy: {scores[1]*100}%")


In [ ]:
model.save('unet_model.h5')


In [ ]:
predictions = model.predict(X_val)
# Convertir las predicciones de one-hot a clases
predicted_classes = np.argmax(predictions, axis=-1)
